In [11]:
# For prerequisites running the following sample, visit https://help.aliyun.com/document_detail/611472.html
from http import HTTPStatus
import dashscope
import os


dashscope.api_key = "sk-db10d05e219d4ba2836874d2b503d1f1"

# 使用同步调用接口

In [30]:
def sample_sync_call(prompt_text):
    resp = dashscope.Generation.call(
        model='qwen-plus',
        prompt=prompt_text
    )
    # The response status_code is HTTPStatus.OK indicate success,
    # otherwise indicate request is failed, you can get error code
    # and message from code and message.
    if resp.status_code == HTTPStatus.OK:
        print(resp.output)  # The output text
        print(resp.usage)  # The usage information
    else:
        print(resp.code)  # The error code.
        print(resp.message)  # The error message.


sample_sync_call('用萝卜、土豆、茄子做饭，给我个菜谱。')

{"text": "这是一个简单的三道菜谱，使用萝卜、土豆和茄子作为主要食材：\n\n1. **红烧茄子**: \n   - 材料：茄子，土豆，胡萝卜，葱姜蒜，酱油，糖，盐，水淀粉\n   - 做法：茄子切段，土豆和胡萝卜切块。热锅凉油，加入糖炒至红色，加入茄子翻煎至微黄。加入葱姜蒜爆香，然后放入土豆和胡萝卜翻炒。倒入酱油，加水没过食材，小火慢炖至软烂，最后用水淀粉勾芡即可。\n\n2. **清炒土豆丝**：\n   - 材料：土豆，胡萝卜，葱，盐，食用油\n   - 做法：土豆和胡萝卜削皮切细丝，葱切末。热锅凉油，油热后加入葱末炒出香味，再加入土豆丝和胡萝卜丝快速翻炒。炒至土豆丝和胡萝卜丝变软，加盐调味，翻炒均匀后出锅。\n\n3. **萝卜炖排骨**：\n   - 材料：排骨，萝卜，生姜，料酒，盐\n   - 做法：排骨焯水去血沫，萝卜切块，生姜切片。锅中加油，下入生姜片炒香，再加入排骨翻煎至两面微黄。加入料酒去腥，然后加水大火烧开，撇去浮沫，转小火慢炖30分钟。加入萝卜块，继续炖煮20分钟，最后加盐调味即可。\n\n这三道菜品结合了炒、炖两种烹饪方式，既有鲜美的红烧茄子，也有清淡的清炒土豆丝和营养丰富的萝卜炖排骨，既健康又美味。", "finish_reason": "stop", "choices": null}
{"input_tokens": 32, "output_tokens": 370, "total_tokens": 402}


# 通过chain + prompt的方式调用

In [31]:
from langchain_community.llms import Tongyi

In [32]:
Tongyi().invoke("What NFL team won the Super Bowl in the year Justin Bieber was born?")

'Justin Bieber was born on March 1, 1994. The Super Bowl that took place in the same year was Super Bowl XXVIII, which was won by the Dallas Cowboys. They defeated the Buffalo Bills with a score of 30-13 on January 30, 1994.'

In [33]:
llm = Tongyi()

from langchain import PromptTemplate

template = """Question: {question}

Answer: """
prompt = PromptTemplate(
    template=template,
    input_variables=['question']
)

chain = prompt | llm
# user question
question = "What NFL team won the Super Bowl in the year Justin Bieber was born?"

chain.invoke({"question": question})

'Justin Bieber was born on March 1, 1994. The Super Bowl that took place in the same year was Super Bowl XXVIII. The winner of that game was the Dallas Cowboys. They defeated the Buffalo Bills with a score of 30-13 on January 30, 1994.'

In [40]:
prompt.format(question="What NFL team won the Super Bowl in the year Justin Bieber was born?")

'Question: What NFL team won the Super Bowl in the year Justin Bieber was born?\n\nAnswer: '

# 流式调用

In [34]:
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_core.messages import HumanMessage

chatLLM = ChatTongyi(
    streaming=True,
)
chatLLM.model_name = 'qwen-plus'
print(chatLLM.model_name)
res = chatLLM.stream([HumanMessage(content="你是千问的哪个模型72b、1.8b还是turbo、plus？")], streaming=True)
for r in res:
    print("chat resp:", r)

qwen-plus
chat resp: content='我是' id='run-95674167-1a72-4bb3-be40-8898ad03cc7a'
chat resp: content='来自' id='run-95674167-1a72-4bb3-be40-8898ad03cc7a'
chat resp: content='阿里' id='run-95674167-1a72-4bb3-be40-8898ad03cc7a'
chat resp: content='云的超大规模语言' id='run-95674167-1a72-4bb3-be40-8898ad03cc7a'
chat resp: content='模型，我叫通义千问' id='run-95674167-1a72-4bb3-be40-8898ad03cc7a'
chat resp: content='。关于我的具体参数量等技术' id='run-95674167-1a72-4bb3-be40-8898ad03cc7a'
chat resp: content='细节，由于涉及敏感信息，我' id='run-95674167-1a72-4bb3-be40-8898ad03cc7a'
chat resp: content='无法提供准确的数值。但可以' id='run-95674167-1a72-4bb3-be40-8898ad03cc7a'
chat resp: content='肯定的是，我是一个经过大量训练' id='run-95674167-1a72-4bb3-be40-8898ad03cc7a'
chat resp: content='和优化的大型预训练模型，' id='run-95674167-1a72-4bb3-be40-8898ad03cc7a'
chat resp: content='能够提供高质量的文本生成和问题' id='run-95674167-1a72-4bb3-be40-8898ad03cc7a'
chat resp: content='解答能力。如果您有任何问题，欢迎' id='run-95674167-1a72-4bb3-be40-8898ad03cc7a'
chat resp: content='随时向我提问。' id='run-95674167-1a72-4bb

# multi role prompt

In [35]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(
        content="You are a helpful assistant that translates English to Chinese."
    ),
    HumanMessage(
        content="Translate this sentence from English to Chinese. I love programming."
    ),
]
chatLLM(messages)

AIMessage(content='我喜欢编程。', response_metadata={'model_name': 'qwen-plus', 'finish_reason': 'stop', 'request_id': '1f6d6235-bd35-93f1-b974-73a634b7679d', 'token_usage': {'input_tokens': 36, 'output_tokens': 3, 'total_tokens': 39}}, id='run-b3ac8ace-bbca-447e-936d-ad1a9e5a626c-0')